In [310]:
# load training data
import pandas as pd
import numpy as np

data_train = pd.read_csv('data_train.csv')
data_test = pd.read_csv('data_test.csv')
train_id = data_train['Id'].astype('int32').values
test_id = data_test['Id'].astype('int32').values
data_train = data_train.drop(['Id'], axis=1)
data_test = data_test.drop(['Id'], axis=1)
# data_train.info()
# data_train.describe()





In [311]:
# handle categorical features and na data

def transform(data):
    data = pd.get_dummies(data, columns=['Модель', 'Марка', 'Тип топлива', 'Трансмиссия','Количество дверей','Количество мест'], dummy_na=True)
    data['Линейный объем']=data['Рабочий объем'] ** 1/3
    data['Ускорение']=data['Мощность двигателя'] ** 1/2
    return data
target = data_train['Цена'].values
data_train = data_train.drop(['Цена'], axis=1)
data_train = transform(data_train)
data_test = transform(data_test)
dtest, dtrain = data_test.align(data_train, join='outer', axis=1)
dtest.info()
dtrain.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74016 entries, 0 to 74015
Columns: 782 entries, Год выпуска to Ускорение
dtypes: float64(66), int64(1), uint8(715)
memory usage: 88.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146012 entries, 0 to 146011
Columns: 782 entries, Год выпуска to Ускорение
dtypes: float64(39), int64(1), uint8(742)
memory usage: 147.9 MB


In [312]:
"""
from sklearn.preprocessing import LabelEncoder

mark = LabelEncoder().fit(data_train['Марка'])
model = LabelEncoder().fit(data_train['Модель'])
fuel = LabelEncoder().fit(data_train['Тип топлива'])
trans = LabelEncoder().fit(data_train['Трансмиссия'])
doors = LabelEncoder().fit(data_train['Количество дверей'])
places = LabelEncoder().fit(data_train['Количество мест'])

def encode_labels(data):
    data['Марка']=mark.transform(data['Марка'].values)
    data['Модель']=model.transform(data['Модель'].values)
    data['Тип топлива']=fuel.transform(data['Тип топлива'].values)
    data['Трансмиссия']=trans.transform(data['Трансмиссия'].values)
    data['Количество мест']=places.transforms(data['Количество мест'].values)
    data['Количество дверей']=doors.transform(data['Количество дверей'].values)
"""

"\nfrom sklearn.preprocessing import LabelEncoder\n\nmark = LabelEncoder().fit(data_train['Марка'])\nmodel = LabelEncoder().fit(data_train['Модель'])\nfuel = LabelEncoder().fit(data_train['Тип топлива'])\ntrans = LabelEncoder().fit(data_train['Трансмиссия'])\ndoors = LabelEncoder().fit(data_train['Количество дверей'])\nplaces = LabelEncoder().fit(data_train['Количество мест'])\n\ndef encode_labels(data):\n    data['Марка']=mark.transform(data['Марка'].values)\n    data['Модель']=model.transform(data['Модель'].values)\n    data['Тип топлива']=fuel.transform(data['Тип топлива'].values)\n    data['Трансмиссия']=trans.transform(data['Трансмиссия'].values)\n    data['Количество мест']=places.transforms(data['Количество мест'].values)\n    data['Количество дверей']=doors.transform(data['Количество дверей'].values)\n"

In [313]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

imp = SimpleImputer(fill_value=0, strategy='constant')
imp.fit(dtrain.values)
sc = StandardScaler()
train=imp.transform(dtrain.values)
sc.fit(train)
#train = sc.transform(train)
test = imp.transform(dtest.values)
#test = sc.transform(test)
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(train)
train = pca.transform(train)
test = pca.transform(test)



In [319]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3)

In [320]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X_train, y_train)
lin_reg.score(X_test, y_test)

0.4453973979342004

In [321]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, n_jobs=10, min_samples_split=10)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

0.5639649421765234


In [322]:
y_pred_lr = lin_reg.predict(test)
y_pred_rf = rf.predict(test)
pd.DataFrame(np.array([test_id, y_pred_lr]).T, columns=['Id','Price'], dtype='int32').to_csv('lin_reg.csv', index=False)
pd.DataFrame(np.array([test_id, y_pred_rf]).T, columns=['Id','Price'], dtype='int32').to_csv('random_forest.csv', index=False)



In [227]:

res.to_csv('lin_reg.csv', index=False)

In [223]:
print(test_id.dtype)

int32
